In [32]:
import sys
import numpy as np
import copy
from functools import lru_cache,wraps
from operator import itemgetter,countOf

import collections
def read_file(file_name):
    with open(file_name) as input_file:
        list_of_lines=input_file.read().splitlines()
    Scanners=[]
    
    for lines in list_of_lines:
        if 'scanner' in lines:
            #print('new scanner')
            scanner=[]
        elif len(lines)>0:
            scanner.append(np.array([int(a) for a in lines.split(',')]))
        else:
            Scanners.append(scanner)
    Scanners.append(scanner)
    return Scanners

def Determ(matrix):
    return matrix[0][0]*matrix[1][1]*matrix[2][2]+\
           matrix[0][1]*matrix[1][2]*matrix[2][0]+\
           matrix[0][2]*matrix[1][0]*matrix[2][1]-\
           matrix[0][0]*matrix[1][2]*matrix[2][1]-\
           matrix[0][1]*matrix[1][0]*matrix[2][2]-\
           matrix[0][2]*matrix[1][1]*matrix[2][0]

def Make_Rotation():
    Matrix=[]
    for i in range(0,24):
        Matrix.append(np.zeros((3,3),np.int64))
        x=i//8
        List=[0,1,2]
        List.remove(x)
        y=List[(i%8)//4]
        Matrix[i][0][x]=(-1)**((i%8)//2)
        Matrix[i][1][y]=(-1)**(i%8)
        List.remove(y)
        z=List[0]
        Matrix[i][2][z]=1
        Matrix[i][2][z]=1/Determ(Matrix[i])                
    return Matrix

def Rotate(Scanners,Matrix):
    ScannersAll=[]
    for i in range(0,len(Scanners)):
        ScannersAll.append([])
        for rot in range(0,24):
            ScannersAll[i].append([ arr@Matrix[rot] for arr in Scanners[i] ])
    return ScannersAll

def Overlap(ScannersAll,n1,n2,Universe,PlacedScanners,ScannersPositions):
    print('Overlap {} {}'.format(n1,n2))
    Source=PlacedScanners[n1]
    Targets=ScannersAll[n2]
    #go through all rotations of Target scanner
    for target in Targets:
        Distances={}
        for p1 in Source:
            for p2 in target:
                if tuple(p1-p2) in Distances:
                    Distances[tuple(p1-p2)] += 1
                else:
                    Distances[tuple(p1-p2)] = 1
        #Compute the largest overlap
        Best_vect=max(Distances, key=Distances.get)
        
        #If more than 12 matc
        if Distances[Best_vect]>=12:
            print('Match found displacement {}, {} matches'.format(Best_vect,Distances[Best_vect]))
            PlaceScanner(target,list(Best_vect),Universe,PlacedScanners)
            ScannersPositions.append(Best_vect)
            return True,n2
    return False,0

def FindMatch(ScannersAll,n1,Universe,PlacedScanners,ScannersPositions):
    ListofMatch=[]
    #Find and place all matching scanners
    for i in range(0,len(ScannersAll)):
        Bool,target=Overlap(ScannersAll,n1,i,Universe,PlacedScanners,ScannersPositions)
        if Bool:
            ListofMatch.append(target)
    #remove all matches from the list of unmatched scanners
    if ListofMatch:
        ListofMatch.sort(reverse=True)
        print('List of matches {}'.format(ListofMatch))
        for i in ListofMatch:
            del(ScannersAll[i])

def PlaceScanner(Scanner,Shift,Universe,PlacedScanners):
    newscan=[]
    for line in Scanner:
        Universe.add(tuple(line+Shift))
        newscan.append(line+Shift)
    PlacedScanners.append(newscan)

if __name__ == '__main__':
    Scanners = read_file('input')
    #Number of Scanners
    NbScanner=len(Scanners)
    print(NbScanner)
    Matrix=Make_Rotation()
    #Position of beacons
    Universe=set()
    #Scanners which have already been placed and rotated
    PlacedScanners=[]

    PlaceScanner(Scanners[0],(0,0,0),Universe,PlacedScanners)
    #List of all scanners with their rotations
    ScannersAll=Rotate(Scanners,Matrix)
    #Scanner 0 has already been placed
    del(ScannersAll[0])
    i=0
    #Position of scanners
    ScannersPositions=[(0,0,0)]
    
    #place all scanners
    while len(PlacedScanners)<NbScanner:
        print('Placed scanner {}'.format(i))
        FindMatch(ScannersAll,i,Universe,PlacedScanners,ScannersPositions)
        i+=1
    print('total number of beacons: {}'.format(len(Universe)))
    max_dist=0
    for i in ScannersPositions:
        for j in ScannersPositions:
            max_dist=max([max_dist,sum(abs(np.array(i)-np.array(j)))])
    print('max distance: {}'.format(max_dist))

25
Placed scanner 0
Overlap 0 0
Overlap 0 1
Overlap 0 2
Overlap 0 3
Overlap 0 4
Overlap 0 5
Match found displacement (1088, -33, 128), 12 matches
Overlap 0 6
Match found displacement (-84, 85, 1346), 12 matches
Overlap 0 7
Overlap 0 8
Overlap 0 9
Overlap 0 10
Overlap 0 11
Overlap 0 12
Overlap 0 13
Overlap 0 14
Overlap 0 15
Match found displacement (-85, -1109, -30), 12 matches
Overlap 0 16
Overlap 0 17
Overlap 0 18
Overlap 0 19
Overlap 0 20
Overlap 0 21
Overlap 0 22
Overlap 0 23
List of matches [15, 6, 5]
4
Placed scanner 1
Overlap 1 0
Overlap 1 1
Match found displacement (2312, 90, 157), 12 matches
Overlap 1 2
Match found displacement (1074, -28, 1220), 12 matches
Overlap 1 3
Overlap 1 4
Overlap 1 5
Overlap 1 6
Overlap 1 7
Overlap 1 8
Overlap 1 9
Overlap 1 10
Overlap 1 11
Overlap 1 12
Overlap 1 13
Overlap 1 14
Overlap 1 15
Overlap 1 16
Overlap 1 17
Overlap 1 18
Overlap 1 19
Overlap 1 20
List of matches [2, 1]
6
Placed scanner 2
Overlap 2 0
Overlap 2 1
Overlap 2 2
Overlap 2 3
Overlap 2